In [12]:
'''Load and Merge the Dataset'''

import pandas as pd 

# Load data

# driver_id, onboarding_date
df_drivers = pd.read_csv('/Users/samirgupte/Library/CloudStorage/OneDrive-Personal/Documents/RPI/JuniorSpring/AI and Blockchain/AI_n_B_Project_Repo/S25_RideChain/Rating_Model/Data/driver_ids.csv')
# ride_id, driver_id, duration, distance, prime_time, etc.
df_rides = pd.read_csv('/Users/samirgupte/Library/CloudStorage/OneDrive-Personal/Documents/RPI/JuniorSpring/AI and Blockchain/AI_n_B_Project_Repo/S25_RideChain/Rating_Model/Data/ride_ids.csv')
# ride_id, timestamp, event (requested, accepted, completed)
df_timestamps = pd.read_csv('/Users/samirgupte/Library/CloudStorage/OneDrive-Personal/Documents/RPI/JuniorSpring/AI and Blockchain/AI_n_B_Project_Repo/S25_RideChain/Rating_Model/Data/ride_timestamps.csv')


# Pivot the timestamp events to wide format: one row per ride_id with columns for each event
df_events = df_timestamps.pivot(index='ride_id', columns='event', values='timestamp').reset_index()

# Convert timestamp columns to datetime
for col in ['requested', 'accepted', 'completed']:
    if col in df_events.columns:
        df_events[col] = pd.to_datetime(df_events[col])

# Joining data on ride_id to obtain per-ride data

# Join ride data with event timestamps
df_combined = pd.merge(df_rides, df_events, on='ride_id', how='inner')

# Join driver info (e.g., onboarding date)
df_combined = pd.merge(df_combined, df_drivers, on='driver_id', how='left')

# Compute accept delay (seconds)
df_combined['accept_delay'] = (pd.to_datetime(df_combined['accepted_at']) - pd.to_datetime(df_combined['requested_at'])).dt.total_seconds()

# Compute completion flag: 1 if the ride was dropped off
df_combined['completed_flag'] = df_combined['dropped_off_at'].notnull().astype(int)

# Preview
print(df_combined[['ride_id', 'driver_id', 'ride_duration', 'ride_distance', 'ride_prime_time', 'accept_delay', 'completed_flag']].head())



                            ride_id                         driver_id  \
0  006d61cf7446e682f7bc50b0f8a5bea5  002be0ffdc997bd5c50703158b7c2491   
1  01b522c5c3a756fbdb12e95e87507eda  002be0ffdc997bd5c50703158b7c2491   
2  029227c4c2971ce69ff2274dc798ef43  002be0ffdc997bd5c50703158b7c2491   
3  034e861343a63ac3c18a9ceb1ce0ac69  002be0ffdc997bd5c50703158b7c2491   
4  034f2e614a2f9fc7f1c2f77647d1b981  002be0ffdc997bd5c50703158b7c2491   

   ride_duration  ride_distance  ride_prime_time  accept_delay  completed_flag  
0            327           1811               50          25.0               1  
1            809           3362                0           3.0               1  
2            572           3282                0           8.0               1  
3           3338          65283               25           4.0               1  
4            823           4115              100           2.0               1  


Plan:

